<a href="https://colab.research.google.com/github/HarshaLokesh/Machine_learning_for_intelligent_vehicles/blob/main/Accident_Severity_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/WASN/Accident_Information.csv")
print(f'Accident_Information.csv : {df.shape}')
df.head(3)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Accident_Information.csv : (2047256, 34)


,Accident_Index,1st_Road_Class,1st_Road_Number,2nd_Road_Class,2nd_Road_Number,Accident_Severity,Carriageway_Hazards,Date,Day_of_Week,Did_Police_Officer_Attend_Scene_of_Accident,Junction_Control,Junction_Detail,Latitude,Light_Conditions,Local_Authority_(District),Local_Authority_(Highway),Location_Easting_OSGR,Location_Northing_OSGR,Longitude,LSOA_of_Accident_Location,Number_of_Casualties,Number_of_Vehicles,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Police_Force,Road_Surface_Conditions,Road_Type,Special_Conditions_at_Site,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Year,InScotland
0,200501BS00001,A,3218.0,NaN,0.0,Serious,None,2005-01-04,Tuesday,1.0,Data missing or out of range,Not at junction or within 20 metres,51.489096,Daylight,Kensington and Chelsea,Kensington and Chelsea,525680.0,178240.0,-0.191170,E01002849,1,1,0.0,1.0,Metropolitan Police,Wet or damp,Single carriageway,None,30.0,17:42,Urban,Raining no high winds,2005,No
1,200501BS00002,B,450.0,C,0.0,Slight,None,2005-01-05,Wednesday,1.0,Auto traffic signal,Crossroads,51.520075,Darkness - lights lit,Kensington and Chelsea,Kensington and Chelsea,524170.0,181650.0,-0.211708,E01002909,1,1,0.0,5.0,Metropolitan Police,Dry,Dual carriageway,None,30.0,17:36,Urban,Fine no high winds,2005,No
2,200501BS00003,C,0.0,NaN,0.0,Slight,None,2005-01-06,Thursday,1.0,Data missing or out of range,Not at junction or within 20 metres,51.525301,Darkness - lights lit,Kensington and Chelsea,Kensington and Chelsea,524520.0,182240.0,-0.206458,E01002857,1,2,0.0,0.0,Metropolitan Police,Dry,Single carriageway,None,30.0,00:15,Urban,Fine no high winds,2005,No


In [ ]:
encoding = {
"Carriageway_Hazards": {"None": 0, "Other object on road": 1, "Any animal in carriageway (except ridden horse)": 1,  "Pedestrian in carriageway - not injured": 1, "Previous accident": 1, "Vehicle load on road": 1,  "Data missing or out of range": 0  }
}
df.replace(encoding, inplace=True)
print(df['Carriageway_Hazards'].value_counts())

0    2010553
1      36703
Name: Carriageway_Hazards, dtype: int64


In [ ]:
print(df['Light_Conditions'].value_counts())
encoding_light = {"Light_Conditions": {"Daylight": 0, "Darkness - lights lit": 1, "Darkness - no lighting": 1, "Darkness - lighting unknown": 1, "Darkness - lights unlit": 1, "Data missing or out of range": 0}}
df.replace(encoding_light, inplace=True)
print(df['Light_Conditions'].value_counts())

Daylight                        1496121
Darkness - lights lit            404144
Darkness - no lighting           112644
Darkness - lighting unknown       24362
Darkness - lights unlit            9971
Data missing or out of range         14
Name: Light_Conditions, dtype: int64
0    1496135
1     551121
Name: Light_Conditions, dtype: int64


In [ ]:
print(df['Day_of_Week'].value_counts())
encoding_day_of_week = {"Day_of_Week": {"Saturday": 1, "Sunday": 1, "Monday": 0, "Tuesday": 0, "Wednesday": 0, "Thursday": 0, "Friday": 0}}
df.replace(encoding_day_of_week, inplace=True)
print(df['Day_of_Week'].value_counts())

Friday       335183
Wednesday    308580
Thursday     308240
Tuesday      306292
Monday       290482
Saturday     273152
Sunday       225327
Name: Day_of_Week, dtype: int64
0    1548777
1     498479
Name: Day_of_Week, dtype: int64


In [ ]:
print(df['Special_Conditions_at_Site'].value_counts())
encoding_Special_Conditions_at_Site = {"Special_Conditions_at_Site": {"None": 0, "Roadworks": 1, "Oil or diesel": 1, "Mud": 1, "Road surface defective": 1, "Auto traffic signal - out": 1, "Road sign or marking defective or obscured": 1, "Auto signal part defective": 1, "Data missing or out of range": 0}}
df.replace(encoding_Special_Conditions_at_Site, inplace=True)
print(df['Special_Conditions_at_Site'].value_counts())

None                                          1995137
Roadworks                                       23525
Oil or diesel                                    6797
Mud                                              6363
Road surface defective                           4801
Auto traffic signal - out                        3855
Road sign or marking defective or obscured       2930
Data missing or out of range                     2835
Auto signal part defective                       1013
Name: Special_Conditions_at_Site, dtype: int64
0    1997972
1      49284
Name: Special_Conditions_at_Site, dtype: int64


In [ ]:
encoding_1st_road_class = {"1st_Road_Class": {"A": 1, "A(M)": 1, "B": 2, "C": 3, "Motorway": 4, "Unclassified": 1}}
df.replace(encoding_1st_road_class, inplace=True)
df['1st_Road_Class'].value_counts()

1    1536156
2     258076
3     174953
4      78071
Name: 1st_Road_Class, dtype: int64

In [ ]:
#replacing 'Data missing or out of range' with most occured value 'Give way or uncontrolled'
df['Junction_Control'] = df['Junction_Control'].replace(['Data missing or out of range'], 'Give way or uncontrolled')

In [ ]:
df['Junction_Control'].value_counts()

Give way or uncontrolled               1742624
Auto traffic signal                     211335
Not at junction or within 20 metres      77304
Stop sign                                12333
Authorised person                         3660
Name: Junction_Control, dtype: int64

In [ ]:
encoding_junction_detail = {"Junction_Control": 
                            {"Give way or uncontrolled": 1,
                             "Auto traffic signal": 2,
                             "Not at junction or within 20 metres": 3, 
                             "Stop sign": 4,
                             "Authorised person": 5,
                              }}
df.replace(encoding_junction_detail, inplace=True)
df['Junction_Control'].value_counts()

1    1742624
2     211335
3      77304
4      12333
5       3660
Name: Junction_Control, dtype: int64

In [ ]:
encoding_junction_detail = {"Junction_Detail": 
                            {"Not at junction or within 20 metres": 1,
                             "T or staggered junction": 2,
                             "Crossroads": 3, 
                             "Roundabout": 4,
                             "Private drive or entrance": 5,
                             "Other junction": 6,
                             "Slip road": 7,
                             "More than 4 arms (not roundabout)": 8,
                             "Mini-roundabout": 9,
                             "Data missing or out of range": 1 }}
df.replace(encoding_junction_detail, inplace=True)
df['Junction_Detail'].value_counts()

1    827957
2    635349
3    196283
4    177214
5     72751
6     59692
7     30052
8     25551
9     22407
Name: Junction_Detail, dtype: int64

In [ ]:
encoding_road_surface_cond = {"Road_Surface_Conditions": 
                            {"Dry": 1,
                             "Wet or damp": 2,
                             "Frost or ice": 3, 
                             "Snow": 4,
                             "Flood over 3cm. deep": 5,
                             "Data missing or out of range": 1 }}
df.replace(encoding_road_surface_cond, inplace=True)
df['Road_Surface_Conditions'].value_counts()

1    1423360
2     568563
3      40321
4      12167
5       2845
Name: Road_Surface_Conditions, dtype: int64

In [ ]:
encoding_road_type = {"Road_Type": 
                            {"Single carriageway": 1,
                             "Dual carriageway": 2,
                             "Roundabout": 3, 
                             "One way street": 4,
                             "Slip road": 5,
                             "Unknown": 0,
                             "Data missing or out of range": 1 }}
df.replace(encoding_road_type, inplace=True)
df['Road_Type'].value_counts()

1    1527883
2     303407
3     136754
4      43258
5      21558
0      14396
Name: Road_Type, dtype: int64

In [ ]:
encoding_urban_rural = {"Urban_or_Rural_Area": 
                            {"Urban": 1,
                             "Rural": 2,
                             "Unallocated": 1 }}
df.replace(encoding_urban_rural, inplace=True)
df['Urban_or_Rural_Area'].value_counts()

1    1322499
2     724757
Name: Urban_or_Rural_Area, dtype: int64

In [ ]:
encoding_weather = {"Weather_Conditions": 
                            {"Fine no high winds": 1,
                             "Raining no high winds": 2,
                             "Raining + high winds": 3,
                             "Fine + high winds": 4,
                             "Snowing no high winds": 5,
                             "Fog or mist": 6,
                             "Snowing + high winds": 7,
                             "Unknown": 1,
                             "Other": 1,
                             "Data missing or out of range": 1 }}
df.replace(encoding_weather, inplace=True)
df['Weather_Conditions'].value_counts()

1    1726874
2     239281
3      28343
4      25816
5      13387
6      11068
7       2487
Name: Weather_Conditions, dtype: int64

In [ ]:
np.where(np.isnan(df['Speed_limit']))

(array([1801605, 1843133, 1843396, 1857338, 1857382, 1857458, 1857466,
        1857525, 1857526, 1857527, 1857531, 1857539, 1857561, 1857564,
        1857583, 1857610, 1857613, 1857618, 1857622, 1857627, 1857635,
        1857681, 1857704, 1857720, 1857736, 1857737, 1857772, 1898106,
        1898251, 1898467, 1898663, 1898938, 1899072, 1899103, 1899306,
        1899388, 1912877]),)

In [ ]:
df['Speed_limit'].fillna((df['Speed_limit'].mean()), inplace=True)

In [ ]:
df['Time'].fillna(0, inplace=True)

In [ ]:
def period(row):
    rdf = []
    if(type(row) == float):
        row = str(row)
        rdf = row.split(".")
    else:
        rdf = str(row).split(":"); # day -- 8am-8pm
        
    hr = rdf[0]
    if int(hr) > 8 and int(hr) < 20:
        return 1;
    else:
        return 2;

In [ ]:
df['Time'] = df['Time'].apply(period)

In [ ]:
df_train1 = df[['1st_Road_Class','Carriageway_Hazards','Junction_Control','Day_of_Week','Junction_Detail','Light_Conditions','Road_Surface_Conditions','Road_Type','Special_Conditions_at_Site','Speed_limit','Time','Urban_or_Rural_Area','Weather_Conditions','Accident_Severity']]

In [ ]:
df_slight = df_train1[df_train1['Accident_Severity']=='Slight']

In [ ]:
df_serious = df_train1[df_train1['Accident_Severity']=='Serious']

In [ ]:
df_fatal = df_train1[df_train1['Accident_Severity']=='Fatal']

In [ ]:
df_serious['Accident_Severity'].value_counts()

Serious    286339
Name: Accident_Severity, dtype: int64

In [ ]:
random_subset = df_slight.sample(n=3)
random_subset.head()

,1st_Road_Class,Carriageway_Hazards,Junction_Control,Day_of_Week,Junction_Detail,Light_Conditions,Road_Surface_Conditions,Road_Type,Special_Conditions_at_Site,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Accident_Severity
1780177,1,0,1,1,1,0,1,1,0,60.0,1,2,1,Slight
1607473,3,0,1,0,1,0,1,4,0,30.0,2,1,1,Slight
208643,1,0,1,0,2,0,2,1,0,30.0,1,1,1,Slight


In [ ]:
df_fatal['Accident_Severity'].value_counts()

Fatal    26369
Name: Accident_Severity, dtype: int64

In [ ]:
df_slight_sampling = df_slight.sample(n=45000)  #Matched the combined number of records for Fatal and Serious(As we are going to club fatal&serious to Serious)

In [ ]:
df_serious_sampling = df_serious.sample(n=24693)  #Matched number of records with the rarer class (Fatal#24693)

In [ ]:
df_final_sampling = pd.concat([df_serious_sampling,df_slight_sampling,df_fatal])

In [ ]:
df_final_sampling.head()

,1st_Road_Class,Carriageway_Hazards,Junction_Control,Day_of_Week,Junction_Detail,Light_Conditions,Road_Surface_Conditions,Road_Type,Special_Conditions_at_Site,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Accident_Severity
2028663,1,0,1,0,1,0,2,1,0,60.0,2,2,2,Serious
1333633,1,0,1,0,1,0,1,1,0,30.0,2,1,1,Serious
267550,1,0,1,0,2,0,1,1,0,40.0,1,1,1,Serious
303448,1,0,1,0,1,0,1,2,0,70.0,1,2,1,Serious
1800871,1,0,1,1,2,0,1,1,0,30.0,2,1,1,Serious


In [ ]:
df_test = df_final_sampling[['Accident_Severity']]

In [ ]:
#replacing 'Data missing or out of range' with most occured value 'None'
df_test['Accident_Severity'] = df_test['Accident_Severity'].replace(['Fatal'], 'Serious')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_train = df_final_sampling[['1st_Road_Class','Carriageway_Hazards','Junction_Control','Day_of_Week','Junction_Detail','Light_Conditions','Road_Surface_Conditions','Road_Type','Special_Conditions_at_Site','Speed_limit','Time','Urban_or_Rural_Area','Weather_Conditions']]

In [ ]:
df_test['Accident_Severity'].value_counts()

Serious    51062
Slight     45000
Name: Accident_Severity, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, df_test, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#class_weight = dict({2:1, 1:15, 0:50})
rdf = RandomForestClassifier(n_estimators=300,random_state=35)

rdf.fit(X_train,y_train)

y_pred=rdf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Accuracy: 0.6103159319210951
[[6086 4049]
 [3438 5640]]
              precision    recall  f1-score   support

     Serious       0.64      0.60      0.62     10135
      Slight       0.58      0.62      0.60      9078

    accuracy                           0.61     19213
   macro avg       0.61      0.61      0.61     19213
weighted avg       0.61      0.61      0.61     19213



In [ ]:
from sklearn.ensemble import RandomForestClassifier
#class_weight = dict({2:1, 1:15, 0:50})
rdf = RandomForestClassifier(bootstrap=True,
            class_weight="balanced_subsample", 
            criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,
            min_samples_leaf=4, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300,
            oob_score=True,
            random_state=35,
            verbose=0, warm_start=False)

In [ ]:
rdf.fit(X_train,y_train)

y_pred=rdf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6288971009212513


In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

[[5687 4448]
 [2682 6396]]
              precision    recall  f1-score   support

     Serious       0.68      0.56      0.61     10135
      Slight       0.59      0.70      0.64      9078

    accuracy                           0.63     19213
   macro avg       0.63      0.63      0.63     19213
weighted avg       0.64      0.63      0.63     19213



In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(learning_rate =0.07, n_estimators=300,
                      class_weight="balanced_subsample",
                      max_depth=8, min_child_weight=1,
                      scale_pos_weight=7,
                      seed=27,subsample=0.8,colsample_bytree=0.8)

model.fit(X_train,y_train)

y_pred=model.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.48311039400405975


In [ ]:
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

[[ 292 9843]
 [  88 8990]]
              precision    recall  f1-score   support

     Serious       0.77      0.03      0.06     10135
      Slight       0.48      0.99      0.64      9078

    accuracy                           0.48     19213
   macro avg       0.62      0.51      0.35     19213
weighted avg       0.63      0.48      0.33     19213



In [ ]:
# import the class
from sklearn.neighbors import KNeighborsClassifier

# instantiate the model (with the default parameters)
knn = KNeighborsClassifier(n_neighbors=3,weights='distance')

# fit the model with data (occurs in-place)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


[[6306 3829]
 [4427 4651]]
              precision    recall  f1-score   support

     Serious       0.59      0.62      0.60     10135
      Slight       0.55      0.51      0.53      9078

    accuracy                           0.57     19213
   macro avg       0.57      0.57      0.57     19213
weighted avg       0.57      0.57      0.57     19213



In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
y_pred = logisticRegr.predict(X_test)
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[[6596 3539]
 [3864 5214]]
              precision    recall  f1-score   support

     Serious       0.63      0.65      0.64     10135
      Slight       0.60      0.57      0.58      9078

    accuracy                           0.61     19213
   macro avg       0.61      0.61      0.61     19213
weighted avg       0.61      0.61      0.61     19213



In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)
print(format(classification_report(y_test, y_pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6112007494925311
              precision    recall  f1-score   support

     Serious       0.63      0.63      0.63     10135
      Slight       0.59      0.59      0.59      9078

    accuracy                           0.61     19213
   macro avg       0.61      0.61      0.61     19213
weighted avg       0.61      0.61      0.61     19213



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(loss="deviance", learning_rate=0.1, 
      n_estimators=100, subsample=1.0, criterion="friedman_mse", 
      min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
      max_depth=3, min_impurity_decrease=0.0, init=None, 
      random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False)

y_pred = gbc.fit(X_train, y_train.values.ravel()).predict(X_test)
print(format(classification_report(y_test, y_pred)))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

     Serious       0.66      0.61      0.64     10135
      Slight       0.60      0.65      0.62      9078

    accuracy                           0.63     19213
   macro avg       0.63      0.63      0.63     19213
weighted avg       0.63      0.63      0.63     19213

0.6287930047363764
